In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Monte Carlo module functions
from src.monte_carlo import (
    mc_baseline,
    mc_antithetic,
    mc_delta_control,
    mc_gamma_control,
    mc_antithetic_delta,
    mc_antithetic_delta_gamma,
    mc_delta,
    mc_gamma,
    mc_vega,
    mc_theta,
    mc_rho,
    compute_greeks as compute_greeks_mc,       # Monte Carlo Greeks
    benchmark_mc_variants
)

# Greeks module
from src.greeks import compare_greeks

# Black-Scholes Greeks
from src.black_scholes import compute_greeks as compute_greeks_bs

# Reproducibility
np.random.seed(42)
sns.set_style("whitegrid")

# Monte Carlo Option Pricing & Greeks Analysis

**Objective:**  
- Implement and analyze Monte Carlo simulation for European option pricing.
- Compare different variance reduction techniques:
  - Antithetic variates
  - Delta control
  - Gamma control
  - Combinations of the above
- Compute and compare option Greeks (Delta, Gamma, Vega, Theta, Rho) using Monte Carlo vs Black-Scholes.
- Quantitatively assess standard error reduction and computational efficiency.

In [ ]:
# Option parameters
S = 101.15      # Underlying asset price
X = 98.01       # Strike price
vol = 0.0991    # Annualized volatility
r = 0.015       # Risk-free rate
T = 60/365      # Time to maturity (years)
type_opt = "C"  # Call option

# Monte Carlo parameters
N = 20          # Time steps
M = 5000        # Simulation paths

# Standard normal random numbers
Z = np.random.normal(size=(N, M))

In [ ]:
# Compare Greeks using Black-Scholes vs Monte Carlo
df_greeks = compare_greeks(S, X, vol, r, N, M, Z, T, type=type_opt)
df_greeks

**Observation:**  
- Monte Carlo estimates should approximate Black-Scholes values within standard error.
- Theta may be slightly noisy due to finite-difference estimation.

In [ ]:
# Compare Monte Carlo variants (prices, SE, computation times)
df_benchmark = benchmark_mc_variants(S, X, vol, r, N, M, Z, T, type=type_opt)
df_benchmark

**Observation:**  
- Variance reduction methods (antithetic, delta, gamma) reduce standard error.
- Examine computation time vs SE trade-offs.

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(x='Variant', y='SE', data=df_benchmark)
plt.title("Monte Carlo Variants: Standard Error Comparison")
plt.ylabel("Standard Error")
plt.show()

In [ ]:
M_values = [1000, 5000, 10000, 20000]
SEs = []

for m in M_values:
    _, SE, _ = mc_antithetic_delta_gamma(S, X, vol, r, N, m, Z[:, :m], T, type_opt)
    SEs.append(SE)

plt.figure(figsize=(8,5))
plt.plot(M_values, SEs, marker='o')
plt.xlabel("Number of Monte Carlo Paths (M)")
plt.ylabel("Standard Error")
plt.title("Convergence of Monte Carlo Standard Error")
plt.show()

In [ ]:
vols = np.linspace(0.05, 0.2, 5)
Ts = np.linspace(30/365, 180/365, 5)
heatmap_data = np.zeros((len(vols), len(Ts)))

for i, vol_i in enumerate(vols):
    for j, T_j in enumerate(Ts):
        _, SE, _ = mc_antithetic_delta_gamma(S, X, vol_i, r, N, M, Z, T_j, type_opt)
        heatmap_data[i, j] = SE

plt.figure(figsize=(8,6))
sns.heatmap(heatmap_data, xticklabels=np.round(Ts,2), yticklabels=np.round(vols,2))
plt.xlabel("Time to Maturity (years)")
plt.ylabel("Volatility")
plt.title("MC SE Heatmap: Volatility vs Maturity")
plt.show()

**Insights:**
- Combined variance reduction (antithetic + delta + gamma) consistently minimizes standard error.
- Increasing the number of paths M decreases SE, but with diminishing returns.
- Monte Carlo Greeks generally match Black-Scholes values within SE.
- Theta is noisier due to finite-difference estimation; larger M/N improves stability.
- This analysis demonstrates quantitative understanding of variance reduction, convergence, and sensitivity of option prices and Greeks.